In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ast
import json
from PIL import Image,ImageDraw,ImageDraw2
import  io
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision
from torchvision import transforms, utils
import torchvision.transforms as T
import os
import cv2
import glob
import time
import tqdm
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
en_dict = {}
dec_dict = {}

In [3]:
def encode_files(csv):
    """ Encode all label by name of csv_files """
    counter = 0
    for fn in csv:
        en_dict[fn[:-4].split('/')[-1].replace(' ', '_')] = counter
        counter += 1
        

def decode_labels(label):
    return dec_dict[label]

def get_label(nfile):
    return en_dict[nfile.replace(' ', '_')[:-4]]

def get_csv(path):
    csv_files = []
    for file in os.listdir(path):
        if file.endswith('csv'):
            csv_files.append(file)
    return csv_files

def strokes_to_arr_1Channel(arr):
    arr = ast.literal_eval(arr)
    x = [x_pnt for stroke in arr for x_pnt in stroke[0]]
    y = [x_pnt for stroke in arr for x_pnt in stroke[1]]

    plt.plot(x,y,color = 'black')
    plt.axis('off')

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.clf()
    
    image = np.array(Image.open(buf))
    buf.close()
    image = np.transpose(image, (2, 0, 1))
    image = 0.2989*image[0] + 0.5870*image[1] + 0.1140*image[2]
    image = np.ceil(image)
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_LINEAR)
    return image


def strokes_to_arr_3Channels(arr):
    arr = ast.literal_eval(arr)
    x = [x_pnt for stroke in arr for x_pnt in stroke[0]]
    y = [x_pnt for stroke in arr for x_pnt in stroke[1]]

    plt.plot(x,y,color = 'black')
    plt.axis('off')

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.clf()
    
    image = np.array(Image.open(buf))
    buf.close()
    image = np.transpose(image, (2, 0, 1))
    image = np.asarray([image[0],image[1],image[2]])
    return image

def validation(lossf, scoref):
    model.eval()
    loss, score = 0, 0
    vlen = len(valid_loader)
    for x, y in valid_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        loss += lossf(output, y).item()
        score += scoref(output, y)[0].item()
    model.train()
    return loss/vlen, score/vlen

def accuracy(output, target, topk=(3,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def mapk(output, target, k=3):
    """
    Computes the mean average precision at k.
    
    Parameters
    ----------
    output (torch.Tensor): A Tensor of predicted elements.
                           Shape: (N,C)  where C = number of classes, N = batch size
    target (torch.int): A Tensor of elements that are to be predicted. 
                        Shape: (N) where each value is  0≤targets[i]≤C−1
    k (int, optional): The maximum number of predicted elements
    
    Returns
    -------
    score (torch.float):  The mean average precision at k over the output
    """
    with torch.no_grad():
        batch_size = target.size(0)

        _, pred = output.topk(k, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        for i in range(k):
            correct[i] = correct[i]*(k-i)
            
        score = correct[:k].view(-1).float().sum(0, keepdim=True)
        score.mul_(1.0 / (k * batch_size))
        return score
    
def squeeze_weights(m):
        m.weight.data = m.weight.data.sum(dim=1)[:,None]
        m.in_channels = 1

In [4]:
csv = get_csv("E:\\Project_FML\\train")
csv = sorted(csv)
encode_files(csv)
dec_dict = {v: k for k, v in en_dict.items()}

## CNN MODEL COMPUTE IS NOT ENOUGH TO TRAIN

In [5]:
# class StrokeDataset(Dataset):
#     def __init__(self, csv_dir, stroke_to_numpy_func, transform=None):
#         self.csv_dir = csv_dir
#         self.transform = transform
#         self.stroke_to_numpy_func = stroke_to_numpy_func
#         self.classes = os.listdir(csv_dir)
#         self.filepaths = []
#         self.labels = []
        
#         for idx, class_name in enumerate(self.classes):
#             csv_file = os.path.join(csv_dir, class_name)
#             df = pd.read_csv(csv_file)
#             for stroke in df['drawing']:
#                 self.filepaths.append(stroke)
#                 self.labels.append(idx)
    
#     def __len__(self):
#         return len(self.filepaths)
    
#     def __getitem__(self, idx):
#         stroke = self.filepaths[idx]
#         label = self.labels[idx]
        
#         # Convert stroke to numpy array and ensure it’s grayscale (224x224)
#         image_np = self.stroke_to_numpy_func(stroke)  # Expected output: (224, 224) for grayscale
        
#         # Resize if needed and convert to tensor
#         transform_resize = T.Resize((224, 224))
#         image = transform_resize(torch.tensor(image_np).unsqueeze(0)).float()  # (1, 224, 224)

#         if self.transform:
#             image = self.transform(image)
        
#         return image, label

# # Define a CNN model for grayscale input
# class CNNModel(nn.Module):
#     def __init__(self, num_classes):
#         super(CNNModel, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Input channels changed to 1
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
#         self.fc1 = nn.Linear(64 * 56 * 56, 256)  # Adjusted for 224x224 input after pooling
#         self.fc2 = nn.Linear(256, num_classes)
    
#     def forward(self, x):
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         x = x.view(-1, 64 * 56 * 56)
#         x = torch.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [6]:
# num_classes = 89
# batch_size = 64
# learning_rate = 0.001
# num_epochs = 10

# dataset = StrokeDataset(csv_dir="/kaggle/input/train-doodle/", stroke_to_numpy_func=strokes_to_arr)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:


# model = CNNModel(num_classes=num_classes).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for images, labels in dataloader:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
        
#         outputs = model(images)
#         loss = criterion(outputs, labels)

#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
    
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

# print("Training complete!")

In [8]:
# model_save_path = 'CNNmodel.pth'
# torch.save(model.state_dict(), model_save_path)
# print(f"Model saved to {model_save_path}")

## TRYING RESNET50 AND RESNET 18

In [15]:
class DoodleDataset(Dataset):
    def __init__(self,csv,directory,function,mode,nrows,transform = None):
        self.csv = csv
        self.directory = directory
        self.function = function
        self.transform = transform
        self.mode = mode 
        self.image = pd.read_csv(directory + csv,usecols = ['drawing'],nrows = nrows)
        self.length = len(self.image)
        self.image = self.image.reset_index()
        self.label = get_label(csv)
        
    def __len__(self):
        return self.length
            
    def __getitem__(self, idx):
        image = self.function(self.image['drawing'][idx])
        label = self.label
    

        # Resize if needed and convert to tensor
        transform_resize = T.Resize((224, 224))
        image = transform_resize(torch.tensor(image).unsqueeze(0)).float()  # (1, 224, 224)
        
        if self.transform:
            image = self.transform(image)
        
        if self.mode == 'train':
            return image, label
        return image

In [16]:
path = "E:\\Project_FML\\train\\"
csv = get_csv(path)


In [17]:
model = torchvision.models.resnet50(pretrained=True)

In [ ]:
dataset = ConcatDataset([DoodleDataset(c,path,strokes_to_arr_1Channel,'train' ,12800,None) for c in csv])
loader = DataLoader(dataset, batch_size=128, shuffle=True,num_workers=0)

In [ ]:
model.conv1.apply(squeeze_weights)

num_classes = 89
model.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, amsgrad=True)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5000,12000,18000], gamma=0.5)

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

Using device: cuda


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
epochs = 1
lsize = len(loader)
itr = 1
p_itr = 1000 # print every N iteration
model.train()
tot = 0
acc = 0
tloss, score = 0, 0
for epoch in range(epochs):
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        _,output = torch.max(output,1)
        loss.backward()
        optimizer.step()
        tloss += loss.item()
        acc += (output == y).sum().item()
        tot += y.size()
        scheduler.step()
        if itr%p_itr==0:
            print('Iteration {} -> Train Loss: {:.4f}, acc: {:.3f}'.format(itr, tloss/p_itr,acc/tot))
            tloss, tot,acc = 0, 0,0
        itr +=1

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
filename_pth='checkpoint_resnet50.pth'
torch.save(model.state_dict(), filename_pth)